In [8]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from datetime import (datetime as dt, timedelta)

# Define date start and end
start_date = dt.now() - timedelta(days=7)
end_date = dt.now()

data = yf.download('BTC-USD', start=start_date, end=end_date, interval='1h')
data.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1, inplace=True)
data.index = data.index.to_pydatetime()

# Função para criar sequências temporais
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequence = data[i : i + seq_length]
        sequences.append(sequence)
    return np.array(sequences)

# Sequencia de valores pretitos
seq_length = 8

prices = data['Adj Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
prices_scaled = scaler.fit_transform(prices)

X = create_sequences(prices_scaled, seq_length)
y = prices_scaled[seq_length:]

# Divindo dados
total_size = len(data)
train_index = int(0.7 * total_size)
test_index = int(0.3 * total_size)

# Dividindo dados
X_train, y_train = X[:train_index], y[:train_index]
X_test, y_test = X[train_index:train_index + test_index], y[train_index:train_index + test_index]

# Construir modelo LSTM
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=(seq_length, 1)),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Treinar o modelo
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Realizar previsões para o conjunto de teste
predictions_scaled_test = model.predict(X_test)

# Desnormalizar as previsões
predictions_test = scaler.inverse_transform(predictions_scaled_test)

# Avaliar o desempenho do modelo
rmse = np.sqrt(mean_squared_error(prices[seq_length + train_index: train_index + test_index + seq_length], predictions_test))
print(f"RMSE: {rmse}")

In [20]:
# # Últimas seq_length horas do conjunto de teste
# last_sequence = prices[-seq_length:].reshape(1, seq_length)

# # Normalizar a última sequência
# last_sequence_scaled = scaler.transform(last_sequence.reshape(-1, 1)).reshape(1, seq_length, 1)

# predictions_scaled_future = []

# for i in range(seq_length):
#     next_timestamp = model.predict(last_sequence_scaled)
#     predictions_scaled_future.append(next_timestamp[0, 0])
    
#     # Atualizar a sequência para incluir a nova previsão e descartar o valor mais antigo
#     last_sequence_scaled = np.roll(last_sequence_scaled, -1, axis=1)
#     last_sequence_scaled[0, -1, 0] = next_timestamp[0, 0]

# predictions_future = scaler.inverse_transform(np.array(predictions_scaled_future).reshape(-1, 1))
# next_24_hours_timestamps = pd.date_range(end=end_date, periods=seq_length, freq='H') + timedelta(hours=1)
# predictions_df = pd.DataFrame({'Timestamp': next_24_hours_timestamps, 'Predicted_Price': predictions_future.flatten()})

In [18]:
# Plotar resultados com plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=prices.flatten(), mode='lines', name='Histórico'))

fig.add_trace(go.Scatter(x=data.index[seq_length + train_index: seq_length + train_index + test_index],
                         y=predictions_test.flatten(),
                         mode='lines',
                         name='Previsões Teste',
                         line=dict(dash='dash', color='green')))

# fig.add_trace(go.Scatter(x=data.index[seq_length + train_index + test_index:],
#                          y=predictions_eval.flatten(),
#                          mode='lines',
#                          name='Previsões Avaliação',
#                          line=dict(dash='dash', color='red')))

# fig.update_layout(title=f'Modelo LSTM - RMSE: ', xaxis_title='Data', yaxis_title='Preço')
fig.update_layout(title=f'Modelo LSTM - RMSE: {rmse:.2f}', xaxis_title='Data', yaxis_title='Preço')
fig.show()
